In [ ]:
import os
import pandas as pd

root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'
dataframes = []

for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)
big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [ ]:
#Normalisation - This reduced accuracy, so no need to run this
column_min_max_values = {
    'angle': (big_dataframe['angle'].min(), big_dataframe['angle'].max()),
    'fsr1': (big_dataframe['fsr1'].min(), big_dataframe['fsr1'].max()),
    'fsr2': (big_dataframe['fsr2'].min(), big_dataframe['fsr2'].max()),
    'fsr3': (big_dataframe['fsr3'].min(), big_dataframe['fsr3'].max()),
    'fsr4': (big_dataframe['fsr4'].min(), big_dataframe['fsr4'].max()),
    'acc2x':(-15.0,15.0),
    'acc2y':(-15.0,15.0),
    'acc2z':(-15.0,15.0),
    'gyro1':(-6.0,6.0),
    'gyro2':(-6.0,6.0),
    'gyro3':(-6.0,6.0),
    #'orient1': (0, 350),
    #'orient2': (0, 100),
    #'orient3': (0, 100),
}
root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'
dataframes = []
for dirpath, dirnames, filenames in os.walk(root_directory):

    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            normalized_data = df.copy()
            for column, (min_val, max_val) in column_min_max_values.items():
                normalized_data[column] = (df[column] - min_val) / (max_val - min_val)
            
            dataframes.append(normalized_data)

big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [ ]:
# This block is to select the 2 out of 4 FSRs for all the participants
grouped_df = big_dataframe.groupby('participant')

participant_column_mapping = {
    1: ['fsr2', 'fsr4'],
    2: ['fsr2', 'fsr4'],
    3: ['fsr2', 'fsr4'],
    4: ['fsr2', 'fsr4'],
    5: ['fsr2', 'fsr4'],
    6: ['fsr2', 'fsr4'],
    9: ['fsr2', 'fsr4'],
    10: ['fsr2', 'fsr3'],
    11: ['fsr2', 'fsr4'],
    12: ['fsr1', 'fsr3'],
    13: ['fsr2', 'fsr4'],
    14: ['fsr2', 'fsr3'],

}
big_dataframe['FSR1'] = ''
big_dataframe['FSR2'] = ''

for participant_id, cols_to_copy in participant_column_mapping.items():
    mask = big_dataframe['participant'] == participant_id
    big_dataframe.loc[mask, 'FSR1'] += big_dataframe.loc[mask, cols_to_copy[0]].astype(str) + ','
    big_dataframe.loc[mask, 'FSR2'] += big_dataframe.loc[mask, cols_to_copy[1]].astype(str) + ','

big_dataframe['FSR1'] = big_dataframe['FSR1'].str.rstrip(',')
big_dataframe['FSR2'] = big_dataframe['FSR2'].str.rstrip(',')

big_dataframe=big_dataframe.drop(columns=['fsr1','fsr2','fsr3','fsr4'])
big_dataframe


In [ ]:
big_dataframe['FSR1'] = big_dataframe['FSR2'].astype(float)
big_dataframe['FSR2'] = big_dataframe['FSR2'].astype(float)

In [ ]:
# Create test set from all participants data

grouped_dataframe = big_dataframe.groupby(['class', 'participant'])

# Function to select 16 random serial numbers and corresponding rows
def select_random_rows(group):
    # Sample 16 random serial numbers
    sampled_serial_numbers = group['serialnumber'].sample(n=17, replace=True, random_state=41).tolist()
    
    # Select all rows with the sampled serial numbers
    selected_rows = group[group['serialnumber'].isin(sampled_serial_numbers)]
    
    return selected_rows


test_set = grouped_dataframe.apply(select_random_rows).reset_index(drop=True)
test_set

In [ ]:
# Create train_set without the data in test_set

# Create a boolean mask to identify rows to be removed
mask = big_dataframe.isin(test_set.to_dict('list')).all(axis=1)
train_set = big_dataframe[~mask].reset_index(drop=True)

In [ ]:
big_dataframe =train_set
test_dataframe = test_set
test_dataframe

In [ ]:
y = big_dataframe.groupby('serialnumber')['class'].first()
y_test = test_dataframe.groupby('serialnumber')['class'].first()
y

In [ ]:
X = big_dataframe.drop(columns=['class','orient1','orient2','orient3','participant'])
test = test_dataframe.drop(columns=['class','orient1','orient2','orient3','participant'])
test

In [ ]:
from tsfresh import extract_features, extract_relevant_features
# Extract features using tsfresh with 'id' column
extracted_features = extract_relevant_features(X,y, column_id='serialnumber', column_sort='time')

In [ ]:
extracted_features

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

num_features_to_retain = 29

# Apply mutual information feature selection
feature_selector = SelectKBest(score_func=mutual_info_regression, k=num_features_to_retain)
selected_features = feature_selector.fit_transform(extracted_features, y)

selected_feature_indices = feature_selector.get_support(indices=True)

# Filter the original feature names to retain only selected features
selected_feature_names = [extracted_features.columns[i] for i in selected_feature_indices]

# Create a DataFrame with the selected features
selected_features_df = extracted_features[selected_feature_names]

In [ ]:
selected_features_df

In [ ]:
selected_features_df = selected_features_df.drop('gyro3__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"', axis=1)
selected_features_df = selected_features_df.drop('gyro3__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"', axis=1)

selected_features_df

In [ ]:
f_feature= pd.Index(["FSR1__cid_ce__normalize_False",'FSR1__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0'])

In [ ]:
import re
import pandas as pd
from tsfresh.feature_extraction import feature_calculators as tsfresh_fc


def generate_code(input_string,sets):
    # Split the string using '__' as the delimiter
    split_parts = re.split("__", input_string)
    
    part1 = split_parts[0]
    part2 = split_parts[1]

    optional_params = {}
    
    for part in split_parts[2:]:

        key, value = part.rsplit("_", 1)
        
        if value.isdigit():
            value = int(value)  # Convert to float if it's a number
        elif value.lower() == "true":
            value = True
        elif value.lower() == "false":
            value = False
        optional_params[key] = value
    
    # Create the code snippet
    code = (
        f"{sets}.groupby('serialnumber')['{part1}'].apply(lambda x: "
        f"tsfresh_fc.{part2}(x, {', '.join([f'{key}={value}' for key, value in optional_params.items()])})).reset_index()"
    )
    
    return code

In [ ]:
new_df = pd.DataFrame()
for feature in f_feature:
    
    code = generate_code(feature,sets="X")
    
    # Execute the code and store the result in a DataFrame
    feature_result = eval(code)
    feature_result.rename(columns={feature_result.columns[-1]: feature}, inplace=True)
    
    if new_df.empty:
        new_df = feature_result
    else:
        new_df = pd.merge(new_df, feature_result, on='serialnumber', how='inner')
new_df.set_index('serialnumber', inplace=True)

selected_features_df=pd.concat([selected_features_df, new_df], axis=1)
selected_features_df

In [ ]:
selected_features_df.columns
final_features = selected_features_df.columns
final_features



In [ ]:
result_df = pd.DataFrame()

# Loop through the selected features
for feature in final_features:
    sets="test"
    # Generate the code for the feature
    if feature == 'gyro3__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"':
        continue
    if feature == 'gyro3__agg_linear_trend__attr_"intercept"__chunk_len___f_agg_"var"': 
        continue
    code = generate_code(feature,sets)

    feature_result = eval(code)
    
    # Rename the column to match the feature name
    feature_result.rename(columns={feature_result.columns[-1]: feature}, inplace=True)

    if result_df.empty:
        result_df = feature_result
    else:
        result_df = pd.merge(result_df, feature_result, on='serialnumber', how='inner')

result_df.set_index('serialnumber', inplace=True)
result_df

In [ ]:
result_df = result_df.dropna()
y_test = y_test.loc[result_df.index]

In [ ]:
X_train = selected_features_df 
y_train = y
X_test = result_df
y_test = y_test

In [ ]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
print(len(y))
print(len(y_test))


In [ ]:
# Decision Tree

import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report  # Add this import
import numpy as np

classifier = DecisionTreeClassifier(max_depth=5)

# Perform k-fold cross-validation
k = 10
accuracy_scores = cross_val_score(classifier, X_train, y_train, cv=k)


average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

# Predict on the test set
dt_y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, dt_y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, dt_y_pred)
print("Classification Report:\n", report)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree


# Visualize the decision tree
plt.figure(figsize=(15, 10))
plot_tree(classifier, filled=True, feature_names=X_train.columns, class_names=True)
plt.show()

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
k = 10
accuracy_scores = cross_val_score(rf_classifier, X_train, y_train, cv=k)
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
rf_classifier.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = rf_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
#Gradient Boost

from sklearn.ensemble import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(n_estimators=92, random_state=42)


k = 10
accuracy_scores = cross_val_score(gb_classifier, X_train, y_train, cv=k)


average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()

gb_classifier.fit(X_train, y_train)


training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = gb_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")


y_pred = gb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

In [ ]:
#Bagging
from sklearn.ensemble import BaggingClassifier
base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Perform k-fold cross-validation
k = 10
accuracy_scores = cross_val_score(bagging_classifier, X_train, y_train, cv=k)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
bagging_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = bagging_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")


y_pred = bagging_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)


In [ ]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier


base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a BaggingClassifier
adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)

# Perform k-fold cross-validation
k = 10
accuracy_scores = cross_val_score(adaboost_classifier, X_train, y_train, cv=k)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()
adaboost_classifier.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Predict on a single instance to measure single prediction time
single_instance = X_test.iloc[[3]] 
start_time_single_prediction = time.time()
single_prediction = adaboost_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")


y_pred = adaboost_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

In [ ]:
#SVM 
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

k = 10
accuracy_scores = cross_val_score(svm_classifier, X_train, y_train, cv=k)


average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()

svm_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

single_instance = X_test.iloc[[3]]
start_time_single_prediction = time.time()
single_prediction = svm_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


In [ ]:
#k-nearest neighbours KNN
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=4)

k = 10
accuracy_scores = cross_val_score(knn_classifier, X_train, y_train, cv=k)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

start_time = time.time()

knn_classifier.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

single_instance = X_test.iloc[[3]] 
start_time_single_prediction = time.time()
single_prediction = knn_classifier.predict(single_instance)

single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time, "seconds")

y_pred = knn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))